In [1]:
import numpy as np
import tensorflow as tf

from deeprte.data import pipeline
from deeprte.model.tf.rte_features import BATCH_FEATURE_NAMES, FEATURE_NAMES
from deeprte.model.tf.rte_features import FEATURES

2023-03-01 12:56:16.255879: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-01 12:56:16.427336: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
"""A Python wrapper for matlab files"""

from pathlib import Path

import numpy as np
import scipy.io as sio
import tree

BATCH_FEAT_LIST = [
    "sigma_a",
    "sigma_t",
    "psi_label",
    "psi_bc",
    "scattering_kernel",
]


def mat_loader(
    source_dir: str, data_name_list: list[str], seed: int = 12345
) -> dict[str, np.ndarray]:

    dir_path = Path(source_dir)
    data_list = []
    for filename in data_name_list:
        data_path = dir_path / filename
        mat_dict = sio.loadmat(data_path)
        if "scattering_kernel" not in mat_dict.keys():
            rng = np.random.default_rng(seed)
            num_sample = mat_dict["sigma_a"].shape[0]
            num_vec = mat_dict["ct"].shape[0]
            mat_dict["scattering_kernel"] = rng.uniform(
                0, 1, (num_sample, num_vec, num_vec)
            )
        data_list.append(mat_dict)
    data = data_list[0]

    for k in BATCH_FEAT_LIST:
        data[k] = np.concatenate([d[k] for d in data_list], axis=0)

    unused_keys = [k for k in data.keys() if k.startswith("__")]
    # for k in unused_keys
    for k in unused_keys:
        del data[k]
    print(mat_dict.keys())
    mat_dict = tree.map_structure(
        lambda x: np.array(x, dtype=np.float32), mat_dict
    )

    return data

In [3]:
source_dir = "../rte_data/matlab/train-scattering-kernel-0204/"
# source_dir = "../rte_data/matlab/eval-data/scattering-kernel/"
# data_name_list = ["test_bc1_scattering_kernel.mat"]
data_name_list = [
    "train_random_kernel_1.mat",
    "train_random_kernel_2.mat",
    "train_random_kernel_3.mat",
    "train_random_kernel_4.mat",
]
data_pipeline = pipeline.DataPipeline(source_dir, data_name_list)
data = data_pipeline.process(
    pre_shuffle=False,
    pre_shuffle_seed=0,
    is_split_test_samples=False,
    num_test_samples=None,
    save_path=None,
)

dict_keys(['__header__', '__version__', '__globals__', 'ct', 'omega', 'omega_prime', 'phi', 'psi_bc', 'psi_label', 'r', 'rv', 'rv_prime', 'scattering_kernel', 'sigma_a', 'sigma_t', 'st', 'w_angle', 'x', 'y'])


In [4]:
for k, v in data.items():
    if isinstance(v, np.ndarray):
        print(f"{k}: {v.dtype}")
    else:
        print(f"{k}: {v}")

sigma: float32
psi_label: float32
scattering_kernel: float32
self_scattering_kernel: float32
boundary: float32
position_coords: float32
velocity_coords: float32
phase_coords: float32
boundary_coords: float32
boundary_weights: float32
velocity_weights: float32
num_x: 41
num_y: 41
num_v: 24
num_examples: 2000
num_train_and_val: 2000


In [5]:
import jax.numpy as jnp
jnp.sum(data["sigma"].astype(int) + data["sigma"], axis=-1).dtype

2023-03-01 12:56:26.178982: W external/org_tensorflow/tensorflow/compiler/xla/service/platform_util.cc:198] unable to create StreamExecutor for CUDA:0: failed initializing StreamExecutor for CUDA device ordinal 0: INTERNAL: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 25438126080


dtype('float32')